<a href="https://colab.research.google.com/github/JRCon1/Options-Pricing/blob/main/Options_Pricing_Methods_(Black_Scholes_Model_%26_Monte_Carlo).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install py_vollib

#Since I am using Google Colab, I need to manually import any atypical packages, and I use the %%capture to hide the output of the package download to avoid clutter

from py_vollib.black_scholes.greeks.analytical import delta, gamma, vega, theta, rho
import pandas as pd
import yfinance as yf
import numpy as np
from datetime import datetime, timedelta
from scipy.stats import norm

r = 0.04 #Predefined a risk-free rate of 0.04. This can be changed here and will remain constant throughout the code
upper = lower = float(input('Enter the upper & lower bound for strikes as a decimal (Ex: 0.20 for 20%): ')) #Enter in the limit of bounds for the strikes as %. IE, if you only want options from 90 to 110 of a $100 stock, pick 0.10, or a 10% move
ticker_symbol = input('Enter the ticker symbol: ').upper() #Pick a ticker/company
opt_type = input('Enter the option type (c for call, p for put): ').lower() #Puts or Calls?
ticker = yf.Ticker(ticker_symbol)
current_price = ticker.history(period="1y")['Close'].iloc[-1]
lower_bound = round(current_price * (1 - lower), 0)
upper_bound = round(current_price * (1 + upper), 0)

#Define function for our baseline filters on the initial Data Query
def filter_opts_within_range(ticker, expiry_dates, lower_bound, upper_bound):
    filtered_options = pd.DataFrame()
    relevant_columns = ['strike', 'lastPrice', 'bid', 'ask', 'volume', 'openInterest', 'impliedVolatility', 'expiry']
    for expiry in expiry_dates:
      if opt_type == 'c':
        calls = ticker.option_chain(expiry).calls
        calls_filtered = calls[(calls['strike'] >= lower_bound) & (calls['strike'] <= upper_bound)].copy()
        calls_filtered['expiry'] = expiry
        filtered_options = pd.concat([filtered_options, calls_filtered[relevant_columns]], ignore_index=True)
      elif opt_type == 'p':
        puts = ticker.option_chain(expiry).puts
        puts_filtered = puts[(puts['strike'] >= lower_bound) & (puts['strike'] <= upper_bound)].copy()
        puts_filtered['expiry'] = expiry
        filtered_options = pd.concat([filtered_options, puts_filtered[relevant_columns]], ignore_index=True)
    return filtered_options

filtered_options = filter_opts_within_range(ticker, ticker.options, lower_bound, upper_bound) #Apply the function
max_expiry_date = datetime.now() + timedelta(weeks=16) #Define max expiry
filtered_options['expiry'] = pd.to_datetime(filtered_options['expiry']) #Apply datetime function to expiry
filtered_options = filtered_options[filtered_options['expiry'] <= max_expiry_date] #Use max_expiry_date variable from previous to filter once again
filtered_options['Days To Expiry'] = (filtered_options['expiry'] - datetime.now()).dt.days + 2 #Define Days to Expiry and adjust for package error
filtered_options['Cost'] = (filtered_options['strike'] - filtered_options['lastPrice']) * 100 #Define cost (for Option Sellers primarily)
filtered_options = filtered_options[filtered_options['Days To Expiry'] > 0] #Get rid of 0 DTE Options (optional)
filtered_options['Moneyness'] = filtered_options['strike'].apply(lambda strike: 'ITM' if (opt_type == 'p' and strike > current_price) or (opt_type == 'c' and strike < current_price) else 'OTM') #Define Moneyness
filtered_options['Mid-Point Price'] = round((filtered_options['bid'] + filtered_options['ask']) / 2, 4) #Define Mid-Point price of Bid and Ask

#Calculate the Intrinsic Value of the Option
def calculate_intrinsic_value(row):
    return 0 if row['Moneyness'] == 'OTM' else row['strike'] - current_price
filtered_options['Intrinsic Value'] = filtered_options.apply(calculate_intrinsic_value, axis=1) * 100 #Apply function and adjust for Net Value
filtered_options['Premium Per Day (Extrinsic)'] = round(((filtered_options['lastPrice'] * 100) - filtered_options['Intrinsic Value']) / filtered_options['Days To Expiry'], 4) #Calculate Premium Per day (Theta Decay over lifetime of option)
filtered_options['Return on Capital as %'] = round((((filtered_options['lastPrice'] * 100) - filtered_options['Intrinsic Value']) / filtered_options['Cost']) * 100, 4) #Return on Capital (For Option Sellers)

#Function to calculate all greeks at once (rounded to the 4th)
def greeks(row, option_type=opt_type, r=r):
    S = current_price
    K = row['strike']
    t = row['Days To Expiry'] / 365
    sigma = row['impliedVolatility']
    delta_value = abs(round(delta(option_type, S, K, t, r, sigma), 4))
    gamma_value = round(gamma(option_type, S, K, t, r, sigma), 4)
    vega_value = round(vega(option_type, S, K, t, r, sigma), 4)
    theta_value = abs(round(theta(option_type, S, K, t, r, sigma), 4))
    rho_value = abs(round(rho(option_type, S, K, t, r, sigma), 4))
    return pd.Series({'Delta': delta_value, 'Gamma': gamma_value, 'Vega': vega_value, 'Theta': theta_value, 'Rho': rho_value})

filtered_options[['Delta', 'Gamma', 'Vega', 'Theta', 'Rho']] = filtered_options.apply(greeks, axis=1) #Apply Functiona for Greeks

Enter the upper & lower bound for strikes as a decimal (Ex: .20 for 20%): 0.2
Enter the ticker symbol: UPRO
Enter the option type (c for call, p for put): p


In [ ]:
#Calculations for Pricing

#Define the Black Scholes Model
def bsm_price(row, option_type='c', r=r):
    S = current_price
    K = row['strike']
    T = row['Days To Expiry'] / 365
    sigma = row['impliedVolatility']
    if T <= 0 or sigma <= 0:
        return 0.0
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'c':
        price = S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'p':
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Option type must be 'c' for call or 'p'.")
    return round(price, 4)

filtered_options['BSM Price'] = filtered_options.apply(bsm_price, axis=1, option_type=opt_type, r=r) #Apply BSM to previous Data Frame

#Define the Monte Carlo Pricing Function
def monte_carlo_price(row, n=10000, r=r, q=0, return_prices=False):
    S = current_price
    K = row['strike']
    T = row['Days To Expiry'] / 365
    sigma = row['impliedVolatility']
    if T <= 0 or sigma <= 0:
        return (0.0, []) if return_prices else 0.0
    Z = np.random.normal(0, 1, n)
    ST = S * np.exp((r - q - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
    if opt_type == 'c':
        payoffs = np.maximum(ST - K, 0)
    elif opt_type == 'p':
        payoffs = np.maximum(K - ST, 0)
    else:
        raise ValueError("Option type must be 'c' for call or 'p'.")
    price = np.exp(-r * T) * np.mean(payoffs)
    return (round(price, 4), ST) if return_prices else round(price, 4)

filtered_options['Monte Carlo Price'] = filtered_options.apply(monte_carlo_price, axis=1) #Apply the Monte Carlo Simulation to the Data Frame

#Return Data to Ensure Accuracy within your preferred range
filtered_options[['strike', 'Mid-Point Price', 'BSM Price', 'Monte Carlo Price', 'Days To Expiry']]

,strike,Mid-Point Price,BSM Price,Monte Carlo Price,Days To Expiry
0,78.0,0.125,0.1237,0.1378,5
1,79.0,0.150,0.1484,0.1438,5
2,80.0,0.075,0.0730,0.0850,5
3,81.0,0.125,0.1228,0.1274,5
4,82.0,0.125,0.1242,0.1224,5
...,...,...,...,...,...
155,96.0,7.300,7.4901,7.3980,96
156,97.0,7.050,6.6761,6.4880,96
157,100.0,8.350,7.8943,7.9492,96
158,105.0,11.200,10.7496,10.7275,96
